In [2]:
import os
import tensorflow as tf
from tensorflow import keras
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
import numpy as np
import json
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix   #copy

train_files = ['../ggt/ggt.npz', '../lp/lp.npz', '../nn/nn.npz']
test_files = ['../zj/zj.npz']

length_8_files = ['../ggt/ggt.npz', '../zj/zj.npz']

train_X = np.concatenate([np.load(i)['X'][:,:,1:] if i in length_8_files else np.load(i)['X'] for i in train_files], axis=0)
train_y = np.concatenate([np.load(i)['y'] for i in train_files], axis=0)

test_X = np.concatenate([np.load(i)['X'][:,:,1:] if i in length_8_files else np.load(i)['X'] for i in test_files], axis=0)
test_y = np.concatenate([np.load(i)['y'] for i in test_files], axis=0)

train_X[np.isnan(train_X)] = 0.
test_X[np.isnan(test_X)] = 0.
train_y = np.array(list(map(lambda x : 1 if x == 5 else 0, train_y)), dtype=np.int32)
test_y = np.array(list(map(lambda x : 1 if x == 5 else 0, test_y)), dtype=np.int32)

results = {}
results['acc'] = []
results['auc'] = []
results['confMat'] = []  #add
fold = 0

index=np.arange(train_X.shape[0])
np.random.shuffle(index)
train_y=train_y[index]
train_X=train_X[index]

#2
model = tf.keras.Sequential([
keras.layers.Masking(mask_value=0., input_shape=(500, 7)), 
keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True), input_shape=(500, 7)),
keras.layers.Bidirectional(keras.layers.LSTM(128)),
keras.layers.Dense(units=64, activation='relu'),
keras.layers.Dense(units=1, activation='sigmoid')
])
#2

model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(from_logits=False), metrics=['accuracy','AUC'])
TensorBoard_callback = keras.callbacks.TensorBoard('logs_without_timestamp/' + str(fold) +"/" , histogram_freq=1)
EarlyStopping_callback = keras.callbacks.EarlyStopping(patience=5)
checkpoint_callback = keras.callbacks.ModelCheckpoint('checkpoint_without_timestamp/fold_' + str(fold) +"/" , save_best_only=True, save_weights_only=True)

model.fit(x=train_X, y=train_y, batch_size=64, epochs=30, validation_split=0.2, callbacks=[TensorBoard_callback, EarlyStopping_callback, checkpoint_callback], verbose=2)

#1
y_pred = model.predict(test_X)
roc = roc_auc_score(test_y, y_pred)
y_pred[y_pred >= 0.5 ] =1
y_pred[y_pred < 0.5] = 0
acc = accuracy_score(test_y, y_pred)
confMat = confusion_matrix(test_y, y_pred).astype(str)
#acc = accuracy_score(np.argmax(test_y, axis=-1), np.argmax(y_pred, axis=-1))
print("In the fold {0}: acc {1}, roc {2}".format(fold, acc, roc))
results['acc'].append(acc)
results['auc'].append(roc)
confMat = [list(A)for A in confMat]
results['confMat'].append(confMat)
fold +=1
#1

with open('auc_acc_without_timestamp.txt','a') as f:
    f.write(json.dumps(results)+ '\n')

Train on 992 samples, validate on 249 samples
Epoch 1/30
992/992 - 30s - loss: 0.4047 - accuracy: 0.8286 - AUC: 0.9021 - val_loss: 0.2424 - val_accuracy: 0.9116 - val_AUC: 0.9680
Epoch 2/30
992/992 - 3s - loss: 0.1439 - accuracy: 0.9506 - AUC: 0.9850 - val_loss: 0.1839 - val_accuracy: 0.9357 - val_AUC: 0.9679
Epoch 3/30
992/992 - 3s - loss: 0.1091 - accuracy: 0.9637 - AUC: 0.9918 - val_loss: 0.1123 - val_accuracy: 0.9719 - val_AUC: 0.9895
Epoch 4/30
992/992 - 3s - loss: 0.0730 - accuracy: 0.9748 - AUC: 0.9964 - val_loss: 0.2137 - val_accuracy: 0.8996 - val_AUC: 0.9842
Epoch 5/30
992/992 - 4s - loss: 0.0901 - accuracy: 0.9738 - AUC: 0.9936 - val_loss: 0.0873 - val_accuracy: 0.9679 - val_AUC: 0.9907
Epoch 6/30
992/992 - 3s - loss: 0.0500 - accuracy: 0.9879 - AUC: 0.9969 - val_loss: 0.0916 - val_accuracy: 0.9759 - val_AUC: 0.9905
Epoch 7/30
992/992 - 3s - loss: 0.0350 - accuracy: 0.9889 - AUC: 0.9989 - val_loss: 0.0934 - val_accuracy: 0.9759 - val_AUC: 0.9923
Epoch 8/30
992/992 - 4s - los